# S3로 이벤트 전송

이 노트북은 매우 간단하며 Amazon Personalize를 위한 사용자 상호 작용을 기반으로 이벤트를 개별 JSON 파일로 S3로 보내는 코드만 제공합니다.

## 설정

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

아래의 캠페인 및 데이터 세트 ARN을 업데이트해야 합니다.

In [ ]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Campaign ARN
Campaign_ARN = "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-demo-camp"

# Dataset Group Arn:
datasetGroupArn = "arn:aws:personalize:us-east-1:059124553121:dataset-group/personalize-lambda-demo"

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')


# S3 Configuration
bucket = "personalizedemo9000chris"

## 가짜 이벤트

이 코드에서는 MovieLens 데이터 세트에 존재하는 것으로 알려진 3명의 사용자를 사용합니다.

1. 196
2. 13
3. 198

이들 사용자는 무작위로 선택되었습니다. 이제 이들 사용자가 상호 작용할 수 있는 3편의 영화를 무작위로 선택하겠습니다.

1. 225
2. 203
3. 242

각 사용자는 각각의 영화와 상호 작용하며, 각 상호 작용은 S3에 고유한 JSON 파일로 기록됩니다.

In [ ]:
users = ["196", "13", "198"]
items = ["225", "203", "242"]

In [ ]:
def write_interaction_to_JSON(user, item):
    # An event needs a user, session, time, type, and a properties dictionary.
    time_of_interaction = int(time.time())
    # This is set for simplification
    session_id = user
    user_id = user
    event_type = "watched"
    properties = {
        'itemId': str(item)
    }
    properties = json.dumps(properties)
    
    event_headers = {
        'userId': user,
        'sessionId': session_id
    }
    
    event_content = {
        'sentAt': time_of_interaction,
        'eventType': event_type,
        'properties': properties
    }
    event_list = [event_content]
    json_struct = [event_headers, event_content]
    json_filename = str(time_of_interaction) + ".json"
    
    with open(json_filename, 'w') as outfile:
        json.dump(json_struct, outfile)
    
    return json_filename

def send_interactions_to_s3():
    for user in users:
        for item in items:
            filename = write_interaction_to_JSON(user, item)
            boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)
            print(filename)
            # Sleep needed to make sure everyone gets a nice new timestamp
            time.sleep(2)
            

In [ ]:
send_interactions_to_s3()

In [ ]:
# The code below is just to explore what to put in our Lambda:

with open("1568076414.json", 'r') as input_file:
    content = json.loads(input_file.read())
    # assume tracking ID was provided via an environment variable and replace
    trackingId = "a68baf70-f0f2-4c66-af2d-e11047a0372f"
    print(content[0]['userId'])
    
    personalize_events.put_events(
        trackingId = trackingId,
        userId = content[0]['userId'],
        sessionId = content[0]['sessionId'],
        eventList = [content[1]]
    )